In [1]:
import nltk
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer, PorterStemmer
from nltk.tokenize import sent_tokenize
from nltk.tokenize import sent_tokenize
import pandas as pd
import re
import string

import csv
df = pd.read_csv("C:/Users/melek/Downloads/large_student_answer_dataset.csv")

# Cümlelere ayırma
# Metin örneğimiz
text = "Gravity keeps us on the ground. It affects all objects on Earth."

# Cümlelere böl
sentences = sent_tokenize(text)

# Lemmatizer ve Stemmer'ı başlat
lemmatizer = WordNetLemmatizer()
stemmer = PorterStemmer()

# Stopwords listesini almak
stop_words = set(stopwords.words('english'))

# Kelimeleri tokenleştirip, lemmatize etme ve stemleme
def preprocess_sentence(sentence):
    tokens = word_tokenize(sentence)  # Cümleyi kelimelere ayır
    # Sadece harf olan kelimeleri al ve stopword'leri çıkar
    filtered_tokens = [token.lower() for token in tokens if token.isalpha() and token.lower() not in stop_words]
    
    lemmatized_tokens = [lemmatizer.lemmatize(token) for token in filtered_tokens]  # Lemmatize etme
    stemmed_tokens = [stemmer.stem(token) for token in filtered_tokens]  # Stemleme
    
    return lemmatized_tokens, stemmed_tokens

# Her cümleyi tokenleştir, lemmatize et ve stemle
tokenized_corpus_lemmatized = []
tokenized_corpus_stemmed = []

for sentence in sentences:
    lemmatized_tokens, stemmed_tokens = preprocess_sentence(sentence)
    tokenized_corpus_lemmatized.append(lemmatized_tokens)
    tokenized_corpus_stemmed.append(stemmed_tokens)

In [2]:
tokenized_corpus_lemmatized[:3]

[['gravity', 'keep', 'u', 'ground'], ['affect', 'object', 'earth']]

In [3]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer

# Ön işlenmiş token listelerini tekrar metne çeviriyoruz
lemmatized_texts = [' '.join(tokens) for tokens in tokenized_corpus_lemmatized]

lemmatized_texts[:3]

['gravity keep u ground', 'affect object earth']

In [4]:
# TF-IDF vektörizerı başlatıyoruz
vectorizer = TfidfVectorizer()

# TF-IDF matrisini oluşturuyoruz
#terim frekansları, belge frekanslarıni hesplar
#TF-IDF vektörlerine dönüştürür
tfidf_matrix = vectorizer.fit_transform(lemmatized_texts)

## Kelimeleri alalım
#F-IDF vektörleştirme işleminde kullanılan tüm kelimelerin essiz bir listesini döndürur
feature_names = vectorizer.get_feature_names_out()

# TF-IDF matrisini pandas DataFrame'e çevir-gorunurluk acisindan- calismasi kolay
tfidf_df = pd.DataFrame(tfidf_matrix.toarray(), columns=feature_names)

# İlk birkaç satırı gösterelim-ilk 5 cümle
print(tfidf_df.head())

#Her satır bir cümleyi temsil eder
#Her sütun bir kelimeyi temsil eder
#Hücreler ise o kelimenin o cümledeki TF-IDF skorudur - her cumle icin degisir-bakiniz:slaytlar

    affect    earth  gravity   ground     keep   object
0  0.00000  0.00000  0.57735  0.57735  0.57735  0.00000
1  0.57735  0.57735  0.00000  0.00000  0.00000  0.57735


In [5]:
# İlk cümle için TF-IDF skorlarını al
first_sentence_vector = tfidf_df.iloc[0]

# Skorlara göre sırala (yüksekten düşüğe)
top_5_words = first_sentence_vector.sort_values(ascending=False).head(5)

# Sonucu yazdır
print("İlk cümlede en yüksek TF-IDF skoruna sahip 5 kelime:")
print(top_5_words)


İlk cümlede en yüksek TF-IDF skoruna sahip 5 kelime:
gravity    0.57735
ground     0.57735
keep       0.57735
affect     0.00000
earth      0.00000
Name: 0, dtype: float64


In [6]:
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Python kelimesinin vektörünü alalım
affect_index = feature_names.tolist().index('affect')  # 'python' kelimesinin indeksini bul

# Python kelimesinin TF-IDF vektörünü alıyoruz ve 2D formatta yapıyoruz
affect_vector = tfidf_matrix[:, affect_index].toarray()

# Tüm kelimelerin TF-IDF vektörlerini alıyoruz
tfidf_vectors = tfidf_matrix.toarray()

# Cosine similarity hesaplayalım
similarities = cosine_similarity(affect_vector.T, tfidf_vectors.T)

# Benzerlikleri sıralayalım ve en yüksek 5 kelimeyi seçelim
similarities = similarities.flatten()
top_5_indices = similarities.argsort()[-6:][::-1]  # 6. en büyükten başlıyoruz çünkü kendisi de dahil

# Sonuçları yazdıralım
for index in top_5_indices:
    print(f"{feature_names[index]}: {similarities[index]:.4f}")

object: 1.0000
earth: 1.0000
affect: 1.0000
keep: 0.0000
ground: 0.0000
gravity: 0.0000
